In [30]:
from langgraph.graph import StateGraph , START , END
from langchain_google_genai import  ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv

In [31]:
class BatsmanState(TypedDict):
    runs: int
    balls: int
    fours: int
    sixes: int
    
    sr: float
    bpb: float
    boundary_percent: float
    summary: str    

In [32]:
def calculate_sr(state: BatsmanState) -> BatsmanState:
    """Calculate the strike rate."""
    sr = (state['runs'] / state['balls']) * 100
    return {'sr': sr}

In [33]:
def calculate_bpb(state: BatsmanState) -> BatsmanState:
    """Calculate the balls per boundary."""
    if state['fours'] + state['sixes'] == 0:
        bpb = float('inf')  # Avoid division by zero
    else:
        bpb = state['balls'] / (state['fours'] + state['sixes'])
    return {'bpb': bpb}

In [34]:
def calculate_boundary_percent(state: BatsmanState) -> BatsmanState:
    """Calculate the boundary percentage."""
    if state['balls'] == 0:
        boundary_percent = 0.0  # Avoid division by zero
    else:
        boundary_percent = (((state['fours']*4) + (state['sixes']*6)) / state['runs']) * 100
    return {'boundary_percent': boundary_percent}

In [35]:
def summary(state: BatsmanState) -> str:
    
    summary = f"""
    Batsman Performance Summary:
    
    Strike Rate: {state['sr']} \n
    Balls Per Boundary: {state['bpb']} \n
    Boundary Percentage: {state['boundary_percent']}
    """
    return {'summary': summary}

In [36]:
graph = StateGraph(BatsmanState)

graph.add_node('calculate_sr' , calculate_sr)
graph.add_node('calculate_bpb' , calculate_bpb)
graph.add_node('calculate_boundary_percent' , calculate_boundary_percent)
graph.add_node('summary' , summary)



#edges 

graph.add_edge(START, 'calculate_sr')
graph.add_edge(START, 'calculate_bpb')
graph.add_edge(START, 'calculate_boundary_percent')

graph.add_edge('calculate_sr', 'summary')
graph.add_edge('calculate_bpb', 'summary')
graph.add_edge('calculate_boundary_percent', 'summary')
 
graph.add_edge('summary', END)


workflow = graph.compile()


In [37]:
initial_state = {
    'runs': 100,
    'balls': 50,
    'fours': 10,
    'sixes': 5,
}

final_state = workflow.invoke(initial_state)
print(final_state)


{'runs': 100, 'balls': 50, 'fours': 10, 'sixes': 5, 'sr': 200.0, 'bpb': 3.3333333333333335, 'boundary_percent': 70.0, 'summary': '\n    Batsman Performance Summary:\n    \n    Strike Rate: 200.0 \n\n    Balls Per Boundary: 3.3333333333333335 \n\n    Boundary Percentage: 70.0\n    '}
